# Tarea de clasificación de texto

En este módulo, comenzaremos con una tarea sencilla de clasificación de texto basada en el conjunto de datos **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: clasificaremos titulares de noticias en una de 4 categorías: Mundo, Deportes, Negocios y Ciencia/Tecnología.

## El Conjunto de Datos

Para cargar el conjunto de datos, utilizaremos la API de **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Ahora podemos acceder a las partes de entrenamiento y prueba del conjunto de datos utilizando `dataset['train']` y `dataset['test']` respectivamente:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Imprimamos los primeros 10 nuevos titulares de nuestro conjunto de datos:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Vectorización de texto

Ahora necesitamos convertir el texto en **números** que puedan representarse como tensores. Si queremos una representación a nivel de palabras, necesitamos hacer dos cosas:

* Usar un **tokenizador** para dividir el texto en **tokens**.
* Construir un **vocabulario** de esos tokens.

### Limitando el tamaño del vocabulario

En el ejemplo del conjunto de datos AG News, el tamaño del vocabulario es bastante grande, más de 100k palabras. En términos generales, no necesitamos palabras que estén presentes raramente en el texto — solo unas pocas frases las tendrán, y el modelo no aprenderá de ellas. Por lo tanto, tiene sentido limitar el tamaño del vocabulario a un número más pequeño pasando un argumento al constructor del vectorizador:

Ambos pasos pueden manejarse utilizando la capa **TextVectorization**. Vamos a instanciar el objeto vectorizador y luego llamar al método `adapt` para recorrer todo el texto y construir un vocabulario:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Nota** que estamos utilizando solo un subconjunto del conjunto de datos completo para construir un vocabulario. Hacemos esto para acelerar el tiempo de ejecución y no hacerte esperar. Sin embargo, corremos el riesgo de que algunas palabras del conjunto de datos completo no se incluyan en el vocabulario y sean ignoradas durante el entrenamiento. Por lo tanto, usar el tamaño completo del vocabulario y procesar todo el conjunto de datos durante `adapt` debería aumentar la precisión final, pero no de manera significativa.

Ahora podemos acceder al vocabulario real:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Usando el vectorizador, podemos codificar fácilmente cualquier texto en un conjunto de números:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Representación de texto con bolsa de palabras

Dado que las palabras representan significado, a veces podemos deducir el significado de un texto simplemente observando las palabras individuales, sin importar su orden en la oración. Por ejemplo, al clasificar noticias, palabras como *clima* y *nieve* probablemente indiquen *pronóstico del tiempo*, mientras que palabras como *acciones* y *dólar* podrían corresponder a *noticias financieras*.

La representación vectorial **bolsa de palabras** (BoW) es la más sencilla de entender entre las representaciones vectoriales tradicionales. Cada palabra está vinculada a un índice de vector, y un elemento del vector contiene el número de veces que aparece cada palabra en un documento dado.

![Imagen que muestra cómo se representa en memoria una representación vectorial de bolsa de palabras.](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png) 

> **Nota**: También puedes pensar en BoW como la suma de todos los vectores codificados en formato one-hot para las palabras individuales en el texto.

A continuación, se muestra un ejemplo de cómo generar una representación de bolsa de palabras utilizando la biblioteca de Python Scikit Learn:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

También podemos usar el vectorizador de Keras que definimos anteriormente, convirtiendo cada número de palabra en una codificación one-hot y sumando todos esos vectores.


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Nota**: Puede que te sorprenda que el resultado difiera del ejemplo anterior. La razón es que, en el ejemplo de Keras, la longitud del vector corresponde al tamaño del vocabulario, que se construyó a partir de todo el conjunto de datos de AG News, mientras que en el ejemplo de Scikit Learn construimos el vocabulario a partir del texto de muestra sobre la marcha.


## Entrenando el clasificador BoW

Ahora que hemos aprendido a construir la representación de bolsa de palabras (bag-of-words) de nuestro texto, entrenemos un clasificador que la utilice. Primero, necesitamos convertir nuestro conjunto de datos a una representación de bolsa de palabras. Esto se puede lograr utilizando la función `map` de la siguiente manera:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Ahora definamos una red neuronal clasificador simple que contiene una capa lineal. El tamaño de entrada es `vocab_size`, y el tamaño de salida corresponde al número de clases (4). Debido a que estamos resolviendo una tarea de clasificación, la función de activación final es **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Dado que tenemos 4 clases, una precisión superior al 80% es un buen resultado.

## Entrenando un clasificador como una sola red

Como el vectorizador también es una capa de Keras, podemos definir una red que lo incluya y entrenarla de principio a fin. De esta manera, no necesitamos vectorizar el conjunto de datos usando `map`, simplemente podemos pasar el conjunto de datos original a la entrada de la red.

> **Nota**: Aún tendríamos que aplicar mapas a nuestro conjunto de datos para convertir campos de diccionarios (como `title`, `description` y `label`) en tuplas. Sin embargo, al cargar datos desde el disco, podemos construir un conjunto de datos con la estructura requerida desde el principio.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigramas, trigramas y n-gramas

Una limitación del enfoque de bolsa de palabras es que algunas palabras forman parte de expresiones de varias palabras. Por ejemplo, la palabra 'hot dog' tiene un significado completamente diferente al de las palabras 'hot' y 'dog' en otros contextos. Si representamos las palabras 'hot' y 'dog' siempre usando los mismos vectores, esto puede confundir a nuestro modelo.

Para abordar este problema, las **representaciones n-gram** se utilizan con frecuencia en métodos de clasificación de documentos, donde la frecuencia de cada palabra, bi-palabra o tri-palabra es una característica útil para entrenar clasificadores. En las representaciones de bigramas, por ejemplo, añadiremos todos los pares de palabras al vocabulario, además de las palabras originales.

A continuación, se muestra un ejemplo de cómo generar una representación de bolsa de palabras con bigramas utilizando Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

La principal desventaja del enfoque de n-gramas es que el tamaño del vocabulario comienza a crecer extremadamente rápido. En la práctica, necesitamos combinar la representación de n-gramas con una técnica de reducción de dimensionalidad, como *embeddings*, que discutiremos en la próxima unidad.

Para usar una representación de n-gramas en nuestro conjunto de datos **AG News**, necesitamos pasar el parámetro `ngrams` al constructor de `TextVectorization`. ¡La longitud de un vocabulario de bigramas es **significativamente mayor**! En nuestro caso, ¡es más de 1.3 millones de tokens! Por lo tanto, tiene sentido limitar también los tokens de bigramas a un número razonable.

Podríamos usar el mismo código que arriba para entrenar el clasificador, sin embargo, sería muy ineficiente en términos de memoria. En la próxima unidad, entrenaremos el clasificador de bigramas utilizando embeddings. Mientras tanto, puedes experimentar con el entrenamiento del clasificador de bigramas en este notebook y ver si puedes obtener una mayor precisión.


## Calculando automáticamente vectores BoW

En el ejemplo anterior calculamos los vectores BoW manualmente sumando las codificaciones one-hot de palabras individuales. Sin embargo, la última versión de TensorFlow nos permite calcular los vectores BoW automáticamente al pasar el parámetro `output_mode='count` al constructor del vectorizador. Esto hace que definir y entrenar nuestro modelo sea significativamente más fácil:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Frecuencia de término - frecuencia inversa de documento (TF-IDF)

En la representación BoW, las ocurrencias de palabras se ponderan utilizando la misma técnica sin importar la palabra en sí. Sin embargo, está claro que palabras frecuentes como *a* e *in* son mucho menos importantes para la clasificación que términos especializados. En la mayoría de las tareas de PLN, algunas palabras son más relevantes que otras.

**TF-IDF** significa **frecuencia de término - frecuencia inversa de documento**. Es una variación de bolsa de palabras, donde en lugar de un valor binario 0/1 que indica la aparición de una palabra en un documento, se utiliza un valor de punto flotante, que está relacionado con la frecuencia de aparición de la palabra en el corpus.

De manera más formal, el peso $w_{ij}$ de una palabra $i$ en el documento $j$ se define como:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
donde
* $tf_{ij}$ es el número de ocurrencias de $i$ en $j$, es decir, el valor de BoW que hemos visto antes
* $N$ es el número de documentos en la colección
* $df_i$ es el número de documentos que contienen la palabra $i$ en toda la colección

El valor TF-IDF $w_{ij}$ aumenta proporcionalmente al número de veces que una palabra aparece en un documento y se ajusta por el número de documentos en el corpus que contienen la palabra, lo que ayuda a compensar el hecho de que algunas palabras aparecen con más frecuencia que otras. Por ejemplo, si la palabra aparece en *todos* los documentos de la colección, $df_i=N$, y $w_{ij}=0$, y esos términos serían completamente ignorados.

Puedes crear fácilmente una vectorización TF-IDF de texto utilizando Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

En Keras, la capa `TextVectorization` puede calcular automáticamente las frecuencias TF-IDF pasando el parámetro `output_mode='tf-idf'`. Repitamos el código que usamos anteriormente para ver si usar TF-IDF aumenta la precisión:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Conclusión

Aunque las representaciones TF-IDF asignan pesos de frecuencia a diferentes palabras, no son capaces de representar el significado ni el orden. Como dijo el famoso lingüista J. R. Firth en 1935: "El significado completo de una palabra siempre es contextual, y ningún estudio del significado fuera del contexto puede tomarse en serio". Más adelante en el curso aprenderemos cómo capturar información contextual del texto utilizando modelos de lenguaje.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
